In [40]:
#import modules 
import numpy as np
from sklearn import preprocessing 

In [41]:
#loading data
raw_data = np.loadtxt("Audiobooks_data.csv", delimiter=",")

#select targest and predictors without ID
inputs_data = raw_data[:,1:-1]
targets_data = raw_data[:,-1]

# since the data was collected with time, it would be better to shuffle before batching 
data_indices = np.arange(inputs_data.shape[0])
np.random.shuffle(data_indices)

ínputs_data = inputs_data[data_indices]
targets_data = targets_data[data_indices]

In [42]:
#balancing data (since the data consist of 1-for converters (those who purchased the audiobook again), 0-for converters), we 
# want to avoid a training data containing more non-converters than converters (already biased datasets)

# sum of targets gives the number of converters 
num_converters = int(np.sum(targets_data))

non_converters_counter = 0
# lets count non-converters 
indices_to_remove = []

for i in range(targets_data.shape[0]):
    if targets_data[i] == 0:
        non_converters_counter +=1
        if non_converters_counter > num_converters:
            indices_to_remove.append(i)
            

inputs_data_equal_priorsis = np.delete(inputs_data, indices_to_remove, axis=0)
targets_data_equal_priors = np.delete(targets_data, indices_to_remove, axis=0)



In [43]:
#standardized dataset
inputs_scaled_data = preprocessing.scale(inputs_data_equal_priorsis)

#shuffle again 
shuffle_indices = np.arange(inputs_scaled_data.shape[0])
np.random.shuffle(shuffle_indices)

inputs_shuffle_data = inputs_scaled_data[shuffle_indices]
targets_shuffle_data = targets_data_equal_priors[shuffle_indices]

In [44]:
#split data into train, validation and test (80%,10%, 10%)

num_samples = inputs_shuffle_data.shape[0]

num_train_samples = int(0.8* num_samples)

num_validation_samples = int(0.1* num_samples)

num_test_samples = num_samples - num_train_samples - num_validation_samples

#selecting data
train_inputs = inputs_shuffle_data[:num_train_samples]
train_targets = targets_shuffle_data[:num_train_samples]

validation_inputs = inputs_shuffle_data[num_train_samples: num_train_samples + num_validation_samples]
validation_targets = targets_shuffle_data[num_train_samples: num_train_samples + num_validation_samples]

test_inputs = inputs_shuffle_data[num_train_samples + num_validation_samples:]
test_targets = targets_shuffle_data[num_train_samples + num_validation_samples:]

# check the number of ones in all the sections 
print(np.sum(train_targets), num_train_samples, np.sum(train_targets)/num_train_samples)
print(np.sum(validation_targets), num_validation_samples, np.sum(validation_targets)/num_validation_samples)
print(np.sum(test_targets), num_test_samples, np.sum(test_targets)/num_test_samples)

1802.0 3579 0.5034925956971221
216.0 447 0.48322147651006714
219.0 448 0.4888392857142857


In [45]:
# save data in npz file 
np.savez("Audiobooks_data_train", inputs= train_inputs, targets=train_targets)
np.savez("Audiobooks_data_validation", inputs= validation_inputs, targets=validation_targets)
np.savez("Audiobooks_data_test", inputs= test_inputs, targets=test_targets)